# naver_api 사용해서 데이터 수집하기

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
client_id=os.getenv("Client_Id")
client_secret=os.getenv("Client_Secret")

print(client_id)
print(client_secret)


EOkeorCGMdv0l8rXYP2W
tOnBCGkcHv


In [46]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request

client_id = os.getenv("Client_Id")
client_secret = os.getenv("Client_Secret") # naver api - id,pw 발급 받고

encText = urllib.parse.quote("삼성전자") #검색어 넣기

url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과 - 블로그 내용 가져옴
#키워드 바꿔주면 다른 곳에서도 가져올 수 있음 (뉴스, 책...)
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과 (문자열+문자열 꼴)
# ? 뒤는 조건 나옴
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    #print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

- 엑셀 형태로 보기 - json 파일 저장해서 dataFrame 만드는 코드 쓰기

In [12]:
response = response_body.decode('utf-8') #문자열 형태로 들어옴

In [13]:
import json

In [43]:
data=json.loads(response) #문자열이 json(딕셔너리) 형태로 바뀜
#data 

In [17]:
type(data)

dict

In [19]:
import pandas as pd

In [42]:
# 수집한 데이터 엑셀(DataFrame)로 정리
result={}

for item in data['items']:
    #print(item)
    for key,value in item.items():
        result.setdefault(key,[]).append(value)

df=pd.DataFrame(result)
#df
        

# naver api의 파라미터 설정 후 최대로 수집하기

파라미터	타입	필수 여부	설명

query : String	Y	검색어. UTF-8로 인코딩되어야 합니다.

display : Integer	N	한 번에 표시할 검색 결과 개수(기본값: 10, 최댓값: 100)

start : Integer	N	검색 시작 위치(기본값: 1, 최댓값: 1000)

sort : String	N	검색 결과 정렬 방법
- sim: 정확도순으로 내림차순 정렬(기본값)
- date: 날짜순으로 내림차순 정렬

In [40]:
import os
import sys
import urllib.request

from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import json
from myfunc import text_clean

# display=100 으로 해서 1page에 100개 자료 수집

In [47]:
# 네이버 검색 API 예제 - 블로그 검색

client_id = os.getenv("Client_Id")
client_secret = os.getenv("Client_Secret")

encText = urllib.parse.quote("삼성전자")

#display=100 으로 해서 1page에 100개 자료 수집
parameters = "&display=100&start=1&sort=sim" #한 페이지에 100개까지 받아서 처리
url = "https://openapi.naver.com/v1/search/blog?query=" + encText+ parameters

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    response = response_body.decode('utf-8')
    data=json.loads(response)
else:
    print("Error Code:" + rescode)
    
# DataFrame으로 정리
result={}
for item in data['items']:
    for key,value in item.items():
        result.setdefault(key,[]).append(value)

df=pd.DataFrame(result)
#df


# start = 1~1000까지 증가, 1page에 100개이므로, 최대 1099개 정도
- 1~100
- 101~200
- 201~300
- 301~400
- 401~500
- 501~600
- 601~700
- 701~800
- 801~900
- 901~1000
- 1000~1100

-> range 함수 이용하기
- range(시작, 끝+1, step)

In [37]:
for num in range(1,1101,100):
    if num>1000:
        num-=1
    print(num)

1
101
201
301
401
501
601
701
801
901
1000


- 이전에 했던 코드 + for문 반복문 코드 합쳐주기

In [49]:
result={}

for num in range(1,1101,100):
    if num>1000:
        num-=1
        
    # 아래 코드 for 반복문 안에 넣기
    client_id = os.getenv("Client_Id")
    client_secret = os.getenv("Client_Secret")

    encText = urllib.parse.quote("삼성전자")

    # 네이버가 제공해준 코드에 파라미터 추가
    parameters = f"&display=100&start={num}&sort=sim" #수정됨! 1100개 자료 모을 수 있음
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText+ parameters

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        response = response_body.decode('utf-8')
        data=json.loads(response)
    else:
        print("Error Code:" + rescode)


    for item in data['items']:
        for key,value in item.items():
            if key in ['title','description']:
                value=text_clean(value) #text_clean 거친 후 추가됨
            result.setdefault(key,[]).append(value)

df=pd.DataFrame(result)
df


,title,link,description,bloggername,bloggerlink,postdate
0,외국인들 삼성전자 팔고 셀트리온 산다는데,https://blog.naver.com/yahoyaho08/224145850607,외국인들은 어떤 종목을 매수하고 있고 우리도 삼성전자 팔고 외국인들이 사는 종목을 ...,디노의 경제 재테크이야기,blog.naver.com/yahoyaho08,20260114
1,물린 사람이 없는 삼성전자주가 0 년 전망은,https://blog.naver.com/goodbye202407/224138808514,0 년을 지나오며 삼성전자 주가를 두고 이런 말이 나옵니다 이 주식 물린 사람이 거...,여행가는 망고주스,blog.naver.com/goodbye202407,20260109
2,홍라희 여사 삼성전자 주식 대량 매도 주가에 악재일까 기회일까,https://blog.naver.com/totoro3123/224151135821,반면 소식을 접하고 의견들을 찾아보니 이번 매각을 삼성전자가 더 높이 날아오르기 위...,너부리의 주식경제노트,blog.naver.com/totoro3123,20260118
3,삼성전자 주식 조원 규모 매도하는 이유는 상속세 및 대출금,https://blog.naver.com/lmj400/224151180213,여러분은 이번 삼성가의 결정을 보며 어떤 생각이 드셨나요 상속세 제도에 대한 견해나...,웰커넥티드,blog.naver.com/lmj400,20260118
4,삼성전자 하이닉스 언제 팔까 나의 매도 기준 가지,https://blog.naver.com/isshoko/224141912142,코스피 00선 돌파라는 역사적 순간의 중심에는 단연 삼성전자와 SK하이닉스가 있습니...,"한녕, 경제로그",blog.naver.com/isshoko,20260110
...,...,...,...,...,...,...
1095,삼성전자 보다 뜨거운 반도체 관련주 TOP 소개,https://blog.naver.com/dkals357/224135146812,대장주인 삼성전자부터 중소형 반도체 장비 부품주까지 전반적인 온기가 확산되고 있기 ...,불개미 현이,blog.naver.com/dkals357,20260105
1096,삼성전자 주가 오르는 이유와 전망 AI 반도체 시대 핵심 기업,https://blog.naver.com/julian1986/224064835229,AI 반도체 수요가 급증하며 삼성전자 주가가 반등하고 있습니다 HBM E 양산 외국...,주택청약과 부동산 공부,blog.naver.com/julian1986,20251105
1097,0대 이상은 삼성전자 0 0은 하이닉스를 사는 이유,https://blog.naver.com/33bsh/224145038701,삼성전자는 이들에게 기회 라기보다 기본값 이다 반면 SK하이닉스를 바라보는 시선은 ...,맥코이의 부동산 투자,blog.naver.com/33bsh,20260113
1098,삼성전자는,https://blog.naver.com/sgtkim79/224148587095,HBM도이제 곧 삼성전자가 메이저 공급업체가 될겁니다 이외에도 삼성전자는 단일기업이...,-정석의 주식투자 일기-,blog.naver.com/sgtkim79,20260116
